# TF-IDF with KNN

In [1]:
# Import library 
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Mengambil data dari csv
# df = pd.read_csv('tf-idf_halodocnew_1.csv')
df = pd.read_csv('df_new_query (1).csv')

print(df.shape)

df.head()

(9979, 20)


,url,title,title_count,content,content_count,title_tokens,content_tokens,title_tokens_stopwords,title_tokens_stopwords_count,content_tokens_stopwords,content_tokens_stopwords_count,title_tokens_stopwords_stemmed,title_tokens_stopwords_stemmed_count,content_tokens_stopwords_stemmed,content_tokens_stopwords_stemmed_count,document_id,new_query,new_query_tokens,new_query_tokens_stopwords,new_query_tokens_stopwords_stemmed
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"ibu, perhatikan 5 hal ini saat memilih car se...",11,"bagi orang tua yang memiliki kendaraan, memili...",575,"['ibu', ',', 'perhatikan', '5', 'hal', 'ini', ...","['bagi', 'orang', 'tua', 'yang', 'memiliki', '...","[',', 'perhatikan', '5', 'memilih', 'car', 'se...",7,"['orang', 'tua', 'memiliki', 'kendaraan', ',',...",339,"['', 'perhati', '5', 'pilih', 'car', 'seat', '...",7,"['orang', 'tua', 'milik', 'kendara', '', 'memi...",339,0,cara memilih seat bayi baru lahir,"['cara', 'memilih', 'seat', 'bayi', 'baru', 'l...","['memilih', 'seat', 'bayi', 'lahir']","['pilih', 'seat', 'bayi', 'lahir']"
1,https://www.halodoc.com/catat-ini-6-tips-menyi...,"catat, ini 6 tips menyimpan beras agar tidak ...",10,sebagai makanan pokok beras harus disimpan den...,617,"['catat', ',', 'ini', '6', 'tips', 'menyimpan'...","['sebagai', 'makanan', 'pokok', 'beras', 'haru...","['catat', ',', '6', 'tips', 'menyimpan', 'bera...",7,"['makanan', 'pokok', 'beras', 'disimpan', 'kua...",347,"['catat', '', '6', 'tips', 'simpan', 'beras', ...",7,"['makan', 'pokok', 'beras', 'simpan', 'kualita...",347,1,NaN,[],[],[]
2,https://www.halodoc.com/waspada-ini-6-bahaya-a...,"waspada, ini 6 bahaya asap rokok bagi kesehat...",10,merokok tidak hanya berbahaya untuk dirimu sen...,708,"['waspada', ',', 'ini', '6', 'bahaya', 'asap',...","['merokok', 'tidak', 'hanya', 'berbahaya', 'un...","['waspada', ',', '6', 'bahaya', 'asap', 'rokok...",8,"['merokok', 'berbahaya', 'dirimu', 'orang-oran...",438,"['waspada', '', '6', 'bahaya', 'asap', 'rokok'...",8,"['rokok', 'bahaya', 'diri', 'orang', 'sekitar'...",438,2,NaN,[],[],[]
3,https://www.halodoc.com/sering-terkena-polusi-...,"sering terkena polusi, ketahui berbagi cara b...",9,"sehari-hari, manusia terpapar polusi dalam jum...",569,"['sering', 'terkena', 'polusi', ',', 'ketahui'...","['sehari-hari', ',', 'manusia', 'terpapar', 'p...","['terkena', 'polusi', ',', 'ketahui', 'berbagi...",7,"['sehari-hari', ',', 'manusia', 'terpapar', 'p...",342,"['kena', 'polusi', '', 'tahu', 'bagi', 'sih', ...",7,"['hari', '', 'manusia', 'papar', 'polusi', 'as...",342,3,NaN,[],[],[]
4,https://www.halodoc.com/ini-risiko-dan-manfaat...,ini risiko dan manfaat implan payudara yang p...,9,implan payudara adalah prosedur bedah kosmetik...,613,"['ini', 'risiko', 'dan', 'manfaat', 'implan', ...","['implan', 'payudara', 'adalah', 'prosedur', '...","['risiko', 'manfaat', 'implan', 'payudara']",4,"['implan', 'payudara', 'prosedur', 'bedah', 'k...",362,"['risiko', 'manfaat', 'implan', 'payudara']",4,"['implan', 'payudara', 'prosedur', 'bedah', 'k...",362,4,NaN,[],[],[]


In [2]:
# Read scraping/halodocnew_1.csv
df_halodocnew_1 = pd.read_csv('scraping/halodocnew_1.csv')
print(len(df_halodocnew_1))

# Ubah kolom "title" dan "content" di df menjadi df_halodocnew_1 (mengembali ke bentuk semula/cased)
df['title'] = df_halodocnew_1['title']
df['content'] = df_halodocnew_1['content']
df.head(1)

9979


,url,title,title_count,content,content_count,title_tokens,content_tokens,title_tokens_stopwords,title_tokens_stopwords_count,content_tokens_stopwords,content_tokens_stopwords_count,title_tokens_stopwords_stemmed,title_tokens_stopwords_stemmed_count,content_tokens_stopwords_stemmed,content_tokens_stopwords_stemmed_count,document_id,new_query,new_query_tokens,new_query_tokens_stopwords,new_query_tokens_stopwords_stemmed
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"Ibu, Perhatikan 5 Hal Ini saat Memilih Car Se...",11,"Bagi orang tua yang memiliki kendaraan, memili...",575,"['ibu', ',', 'perhatikan', '5', 'hal', 'ini', ...","['bagi', 'orang', 'tua', 'yang', 'memiliki', '...","[',', 'perhatikan', '5', 'memilih', 'car', 'se...",7,"['orang', 'tua', 'memiliki', 'kendaraan', ',',...",339,"['', 'perhati', '5', 'pilih', 'car', 'seat', '...",7,"['orang', 'tua', 'milik', 'kendara', '', 'memi...",339,0,cara memilih seat bayi baru lahir,"['cara', 'memilih', 'seat', 'bayi', 'baru', 'l...","['memilih', 'seat', 'bayi', 'lahir']","['pilih', 'seat', 'bayi', 'lahir']"


In [3]:
# Melakukan konversi data yang sebelumnya bertipe string agar bisa diolah pada tahap selanjutnya

import ast

df['title_tokens'] = df['title_tokens'].apply(ast.literal_eval)
df['content_tokens'] = df['content_tokens'].apply(ast.literal_eval)
df['title_tokens_stopwords'] = df['title_tokens_stopwords'].apply(ast.literal_eval)
df['content_tokens_stopwords'] = df['content_tokens_stopwords'].apply(ast.literal_eval)
df['content_tokens_stopwords_stemmed'] = df['content_tokens_stopwords_stemmed'].apply(ast.literal_eval)
df['title_tokens_stopwords_stemmed'] = df['title_tokens_stopwords_stemmed'].apply(ast.literal_eval)


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Mendefiniskan konfigurasi untuk Nearest Neighbors
retriever_configs = {
    'n_neighbors': 9,  # Jumlah tetangga/dokumen terdekat yang akan dicari
    'metric': 'cosine'  # Fungsi similarity yang akan digunakan
}

# Mendefinisikan pipeline untuk sistem Information Retrieval
# menggunakan TF-IDF dan Nearest Neighbors (dengan metrik cosine similarity)
embedding = TfidfVectorizer()
retriever = NearestNeighbors(**retriever_configs)

In [5]:
# Membuat kolom baru untuk indeks dokumen
df['document_id'] = range(len(df))

In [6]:
# Melakukan word embedding pada content_tokens_stopwords_stemmed (dokumen/konteks) menggunakan TF-IDF
X = embedding.fit_transform(df['content_tokens_stopwords_stemmed'].apply(lambda tokens: ' '.join(tokens)))

# Melakukan fit/training pada NearestNeighbors
retriever.fit(X, df['document_id'])

NearestNeighbors(metric='cosine', n_neighbors=9)

In [7]:
# Question/pertanyaan yang akan digunakan sebagai input 'query
question = 'tips untuk ibu hamil'

In [8]:
# Fungsi untuk melakukan transformasi teks menjadi vektor
def transform_text(vectorizer, text):
    print('Text:', text)
    vector = vectorizer.transform([text])
    vector = vectorizer.inverse_transform(vector)
    print('Vect:', vector)

# Melakukan transformasi teks menjadi vektor dengan fungsi transform_text
transform_text(embedding, question)

Text: tips untuk ibu hamil
Vect: [array(['untuk', 'tips', 'ibu', 'hamil'], dtype='<U72')]


In [9]:
# Melakukan stemming pada query (question/pertanyaan yang dimasukkan) menggunakan Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = set(stopwords.words('indonesian'))

# Menghilangkan stopwords dan melakukan stemming pada query 
question = [stemmer.stem(word) for word in word_tokenize(question) if word not in stop_words]
question = ' '.join(question)


# Memperoleh (prediksi) dokumen yang paling mirip dengan query
X = embedding.transform([question])
document_id = retriever.kneighbors(X, return_distance=False)[0][0]
selected = df.iloc[document_id]['content']

selected

'Kehamilan dapat menjadi momen yang dinantikan oleh beberapa pasangan. Pada beberapa kesempatan, kehamilan dapat menambah beban pada sang ibu, terlebih jika sudah memasuki trimester akhir. Lalu, apa saja sih tips untuk memiliki masa kehamilan yang nyaman? Ketahui jawabannya di sini! Tips untuk Mendapatkan Masa Kehamilan yang Nyaman Rasa gembira memiliki bayi bisa saja mereda karena berbagai masalah fisik yang timbul. Ibu bisa saja mengalami rasa sakit, nyeri, atau ketidaknyamanan ketika hamil. Hal ini terjadi karena adanya perubahan pada tubuh selama kehamilan. Meski begitu, tidak semua wanita mengalami masalah ini ketika hamil.  Namun, saat mengalaminya\xa0 ibu perlu tahu beberapa tips untuk mendapatkan masakehamilanyang nyaman, yaitu: 1.\xa0 Rutin Memeriksakan Kehamilan Tips kehamilan yang pertama adalah secara rutin mengontrol ke dokter untuk memeriksakan kehamilan. Bahkan pemeriksaan harus lebih sering dilakukan di trimester akhir kehamilan. Selain itu, jangan ragu untuk segera men

In [10]:
# Melakukan transformasi pada dokumen yang dipilih dengan fungsi yang telah dibuat sebelumnya
transform_text(embedding, selected)

Text: Kehamilan dapat menjadi momen yang dinantikan oleh beberapa pasangan. Pada beberapa kesempatan, kehamilan dapat menambah beban pada sang ibu, terlebih jika sudah memasuki trimester akhir. Lalu, apa saja sih tips untuk memiliki masa kehamilan yang nyaman? Ketahui jawabannya di sini! Tips untuk Mendapatkan Masa Kehamilan yang Nyaman Rasa gembira memiliki bayi bisa saja mereda karena berbagai masalah fisik yang timbul. Ibu bisa saja mengalami rasa sakit, nyeri, atau ketidaknyamanan ketika hamil. Hal ini terjadi karena adanya perubahan pada tubuh selama kehamilan. Meski begitu, tidak semua wanita mengalami masalah ini ketika hamil.  Namun, saat mengalaminya  ibu perlu tahu beberapa tips untuk mendapatkan masakehamilanyang nyaman, yaitu: 1.  Rutin Memeriksakan Kehamilan Tips kehamilan yang pertama adalah secara rutin mengontrol ke dokter untuk memeriksakan kehamilan. Bahkan pemeriksaan harus lebih sering dilakukan di trimester akhir kehamilan. Selain itu, jangan ragu untuk segera mene

## Query unpreprocessed

In [11]:

# X_ev = embedding.transform(df['title_tokens_stopwords_stemmed'].apply(lambda tokens: ' '.join(tokens)))
X_ev = embedding.transform(df['title'])
X_ev.shape

(9979, 61940)

In [12]:
y_ev = df['document_id']
y_ev.shape

(9979,)

In [13]:
y_pred = retriever.kneighbors(X_ev, return_distance=False)
print(y_pred.shape)
y_pred

(9979, 9)


array([[   0, 3073, 7718, ..., 3553, 5276, 9826],
       [6513,  278, 6758, ..., 6237, 3365,    1],
       [   2, 8838, 7095, ..., 8007, 8715, 6664],
       ...,
       [9976, 9971, 3505, ...,   37, 5982, 6605],
       [1378, 2413, 3765, ..., 6792, 5404, 7542],
       [9978, 4562,  921, ...,  308, 9251, 7837]], dtype=int64)

In [14]:
def top_accuracy(y_true, y_pred) -> float:
    right, count = 0, 0
    for i, y_t in enumerate(y_true):
        count += 1
        if y_t in y_pred[i]:
            right += 1
    return right / count if count > 0 else 0

In [15]:
acc = top_accuracy(y_ev, y_pred)
print('Jumlah Benar\t:', int(acc*len(y_ev)), 'dari', len(y_ev))
print('Akurasi\t\t:', f'{acc:.4f}')

Jumlah Benar	: 8572 dari 9979
Akurasi		: 0.8590


## Query preprocessed

In [16]:

X_ev = embedding.transform(df['title_tokens_stopwords_stemmed'].apply(lambda tokens: ' '.join(tokens)))
y_ev = df['document_id']
y_pred = retriever.kneighbors(X_ev, return_distance=False)
print(y_pred.shape)


acc = top_accuracy(y_ev, y_pred)
print('Jumlah Benar\t:', int(acc*len(y_ev)), 'dari', len(y_ev))
print('Akurasi\t\t:', f'{acc:.4f}')

(9979, 9)
Jumlah Benar	: 9338 dari 9979
Akurasi		: 0.9358


# Evaluate with alternative query from doc2query

In [17]:
# df = pd.read_csv('df_new_query (1).csv')

df['new_query_tokens_stopwords_stemmed'] = df['new_query_tokens_stopwords_stemmed'].apply(ast.literal_eval)
additional_queries = df['new_query_tokens_stopwords_stemmed'].apply(lambda tokens: ' '.join(tokens) if len(tokens) != 0 else '')
additional_queries = pd.DataFrame({'query': additional_queries, 'document_id': df['document_id']})
additional_queries.head()

,query,document_id
0,pilih seat bayi lahir,0
1,,1
2,,2
3,,3
4,,4


In [18]:
# drop null values (70% of the data)
# Menghapus baris yang memiliki array kosong pada kolom "query"
empty_query_rows = additional_queries[additional_queries['query'].apply(lambda x: len(x) == 0)]
additional_queries = additional_queries.drop(empty_query_rows.index)
additional_queries = additional_queries.reset_index(drop=True)
print(additional_queries.shape)
additional_queries.head()

(2993, 2)


,query,document_id
0,pilih seat bayi lahir,0
1,allylestrenol aman hamil,5
2,tes urine,6
3,obat obat sindrom orang kaku,15
4,jenis alat ukur suhu,21


In [19]:
# additional_queries['query'] = additional_queries['query'].apply(ast.literal_eval)
# additional_queries['query'].apply(lambda tokens: ' '.join(tokens))

In [20]:
X_ev_doc2query = embedding.transform(additional_queries['query'])
y_ev_doc2query = additional_queries['document_id']
y_pred_doc2query = retriever.kneighbors(X_ev_doc2query, return_distance=False)
print(y_pred_doc2query.shape)


(2993, 9)


In [21]:
y_pred_doc2query

array([[   0, 7271, 6119, ..., 7017, 9324, 3387],
       [   5, 7861, 7814, ..., 2199, 9828, 9768],
       [   6, 1857, 1626, ..., 4427, 1056, 5145],
       ...,
       [9898,  211, 9968, ..., 9060, 9922, 2228],
       [7519, 9974, 7449, ..., 2112, 7861, 2659],
       [9976, 8745, 7924, ..., 6605, 1371, 9971]], dtype=int64)

In [22]:
acc = top_accuracy(y_ev_doc2query, y_pred_doc2query)
print('Jumlah Benar\t:', int(acc*len(y_ev_doc2query)), 'dari', len(y_ev_doc2query))
print('Akurasi\t\t:', f'{acc:.4f}')

Jumlah Benar	: 2482 dari 2993
Akurasi		: 0.8293


In [46]:
retriever

NearestNeighbors(metric='cosine', n_neighbors=9)

# New Dataset

In [23]:
# read tf-idf_train.csv
df_json = pd.read_csv('tf-idf_train.csv')
df_json.head(1)

,id,text,label,text_word_count,label_word_count,text_lower,label_lower,text_tokens,label_tokens,text_tokens_stopwords,label_tokens_stopwords,text_tokens_stopwords_stemmed,label_tokens_stopwords_stemmed
0,1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"Dokter Ryan Thamrin , yang terkenal lewat aca...",Dokter Lula Kamal yang merupakan selebriti sek...,330,58,"dokter ryan thamrin , yang terkenal lewat aca...",dokter lula kamal yang merupakan selebriti sek...,"['dokter', 'ryan', 'thamrin', ',', 'yang', 'te...","['dokter', 'lula', 'kamal', 'yang', 'merupakan...","['dokter', 'ryan', 'thamrin', ',', 'terkenal',...","['dokter', 'lula', 'kamal', 'selebriti', 'reka...","['dokter', 'ryan', 'thamrin', '', 'kenal', 'ac...","['dokter', 'lula', 'kamal', 'selebriti', 'reka..."


In [24]:
# ast literal on text_tokens_stopwords_stemmed and label_tokens_stopwords_stemmed
df_json['text_tokens_stopwords_stemmed'] = df_json['text_tokens_stopwords_stemmed'].apply(ast.literal_eval)
df_json['label_tokens_stopwords_stemmed'] = df_json['label_tokens_stopwords_stemmed'].apply(ast.literal_eval)
df_json.head(1)

,id,text,label,text_word_count,label_word_count,text_lower,label_lower,text_tokens,label_tokens,text_tokens_stopwords,label_tokens_stopwords,text_tokens_stopwords_stemmed,label_tokens_stopwords_stemmed
0,1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"Dokter Ryan Thamrin , yang terkenal lewat aca...",Dokter Lula Kamal yang merupakan selebriti sek...,330,58,"dokter ryan thamrin , yang terkenal lewat aca...",dokter lula kamal yang merupakan selebriti sek...,"['dokter', 'ryan', 'thamrin', ',', 'yang', 'te...","['dokter', 'lula', 'kamal', 'yang', 'merupakan...","['dokter', 'ryan', 'thamrin', ',', 'terkenal',...","['dokter', 'lula', 'kamal', 'selebriti', 'reka...","[dokter, ryan, thamrin, , kenal, acara, dokter...","[dokter, lula, kamal, selebriti, rekan, kerja,..."


In [25]:
# change df_json columns name
#  - text -> content
#  - label -> title
#  - text_tokens_stopwords_stemmed -> content_tokens_stopwords_stemmed
#  - label_tokens_stopwords_stemmed -> title_tokens_stopwords_stemmed

df_json = df_json.rename(columns={'text': 'content', 'label': 'title', 'text_tokens_stopwords_stemmed': 'content_tokens_stopwords_stemmed', 'label_tokens_stopwords_stemmed': 'title_tokens_stopwords_stemmed'})
df_json.head(1)

,id,content,title,text_word_count,label_word_count,text_lower,label_lower,text_tokens,label_tokens,text_tokens_stopwords,label_tokens_stopwords,content_tokens_stopwords_stemmed,title_tokens_stopwords_stemmed
0,1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"Dokter Ryan Thamrin , yang terkenal lewat aca...",Dokter Lula Kamal yang merupakan selebriti sek...,330,58,"dokter ryan thamrin , yang terkenal lewat aca...",dokter lula kamal yang merupakan selebriti sek...,"['dokter', 'ryan', 'thamrin', ',', 'yang', 'te...","['dokter', 'lula', 'kamal', 'yang', 'merupakan...","['dokter', 'ryan', 'thamrin', ',', 'terkenal',...","['dokter', 'lula', 'kamal', 'selebriti', 'reka...","[dokter, ryan, thamrin, , kenal, acara, dokter...","[dokter, lula, kamal, selebriti, rekan, kerja,..."


In [27]:
# combine df_json and df to df_combined
#  - df == df_json (was)
# - 'content' = 'text'
# - 'title' = 'label'
# - 'content_tokens_stopwords_stemmed' = 'text_tokens_stopwords_stemmed'
# - 'title_tokens_stopwords_stemmed' = 'label_tokens_stopwords_stemmed'

columns = ['content', 'title', 'content_tokens_stopwords_stemmed', 'title_tokens_stopwords_stemmed']

# make empty df_combined
df_combined = pd.DataFrame(columns=columns)

# Menggabungkan df dan df_json menjadi df_combined dengan kolom 'content', 'title', 'content_tokens_stopwords_stemmed', 'title_tokens_stopwords_stemmed'
df_combined = pd.concat([df, df_json[columns]], ignore_index=True)
df_combined = df_combined.drop(df_combined.columns.difference(columns), axis=1)
df_combined

,title,content,title_tokens_stopwords_stemmed,content_tokens_stopwords_stemmed
0,"Ibu, Perhatikan 5 Hal Ini saat Memilih Car Se...","Bagi orang tua yang memiliki kendaraan, memili...","[, perhati, 5, pilih, car, seat, newborn]","[orang, tua, milik, kendara, , memilihcar, sea..."
1,"Catat, Ini 6 Tips Menyimpan Beras agar Tidak ...",Sebagai makanan pokok beras harus disimpan den...,"[catat, , 6, tips, simpan, beras, kutu]","[makan, pokok, beras, simpan, kualitas, turun,..."
2,"Waspada, Ini 6 Bahaya Asap Rokok bagi Kesehat...",Merokok tidak hanya berbahaya untuk dirimu sen...,"[waspada, , 6, bahaya, asap, rokok, sehat, anak]","[rokok, bahaya, diri, orang, sekitar, , bayi, ..."
3,"Sering Terkena Polusi, Ketahui Berbagi Cara B...","Sehari-hari, manusia terpapar polusi dalam jum...","[kena, polusi, , tahu, bagi, sih, paru]","[hari, , manusia, papar, polusi, asap, kendara..."
4,Ini Risiko dan Manfaat Implan Payudara yang P...,Implan payudara adalah prosedur bedah kosmetik...,"[risiko, manfaat, implan, payudara]","[implan, payudara, prosedur, bedah, kosmetik, ..."
...,...,...,...,...
24057,Filosofi inilah yang menjadi pegangan bagi ana...,JANGAN engkau terus meratapi nasib hanya karen...,"[filosofi, pegang, anak, delapan, puluh, sauda...","[engkau, ratap, nasib, takdir, lahir, keluarga..."
24058,"Jakarta , CNN Indonesia - - Aktris Lindsay L...",Aktris Lindsay Lohan mencuri perhatian publ...,"[jakarta, , cnn, indonesia, -, -, aktris, lind...","[aktris, lindsay, loh, curi, perhati, publik, ..."
24059,Jakarta ( ANTARA News ) - Doktor Biomedik dari...,Doktor Biomedik dari Fakultas Kedokteran Unive...,"[jakarta, , news, , -, doktor, biomedik, fakul...","[doktor, biomedik, fakultas, dokter, universit..."
24060,"Jakarta , CNN Indonesia - - Tim peneliti Indon...",Tim peneliti Indonesia yang bergabung dalam...,"[jakarta, , cnn, indonesia, -, -, tim, teliti,...","[tim, teliti, indonesia, gabung, murdoch, chil..."


In [28]:
# Drop null values
df_combined = df_combined.dropna()
df_combined = df_combined.reset_index(drop=True)
print(len(df_combined))

24060


In [29]:
# Tambahkan kolom 'document_id' pada df_combined
df_combined['document_id'] = df_combined.index
df_combined

,title,content,title_tokens_stopwords_stemmed,content_tokens_stopwords_stemmed,document_id
0,"Ibu, Perhatikan 5 Hal Ini saat Memilih Car Se...","Bagi orang tua yang memiliki kendaraan, memili...","[, perhati, 5, pilih, car, seat, newborn]","[orang, tua, milik, kendara, , memilihcar, sea...",0
1,"Catat, Ini 6 Tips Menyimpan Beras agar Tidak ...",Sebagai makanan pokok beras harus disimpan den...,"[catat, , 6, tips, simpan, beras, kutu]","[makan, pokok, beras, simpan, kualitas, turun,...",1
2,"Waspada, Ini 6 Bahaya Asap Rokok bagi Kesehat...",Merokok tidak hanya berbahaya untuk dirimu sen...,"[waspada, , 6, bahaya, asap, rokok, sehat, anak]","[rokok, bahaya, diri, orang, sekitar, , bayi, ...",2
3,"Sering Terkena Polusi, Ketahui Berbagi Cara B...","Sehari-hari, manusia terpapar polusi dalam jum...","[kena, polusi, , tahu, bagi, sih, paru]","[hari, , manusia, papar, polusi, asap, kendara...",3
4,Ini Risiko dan Manfaat Implan Payudara yang P...,Implan payudara adalah prosedur bedah kosmetik...,"[risiko, manfaat, implan, payudara]","[implan, payudara, prosedur, bedah, kosmetik, ...",4
...,...,...,...,...,...
24055,Filosofi inilah yang menjadi pegangan bagi ana...,JANGAN engkau terus meratapi nasib hanya karen...,"[filosofi, pegang, anak, delapan, puluh, sauda...","[engkau, ratap, nasib, takdir, lahir, keluarga...",24055
24056,"Jakarta , CNN Indonesia - - Aktris Lindsay L...",Aktris Lindsay Lohan mencuri perhatian publ...,"[jakarta, , cnn, indonesia, -, -, aktris, lind...","[aktris, lindsay, loh, curi, perhati, publik, ...",24056
24057,Jakarta ( ANTARA News ) - Doktor Biomedik dari...,Doktor Biomedik dari Fakultas Kedokteran Unive...,"[jakarta, , news, , -, doktor, biomedik, fakul...","[doktor, biomedik, fakultas, dokter, universit...",24057
24058,"Jakarta , CNN Indonesia - - Tim peneliti Indon...",Tim peneliti Indonesia yang bergabung dalam...,"[jakarta, , cnn, indonesia, -, -, tim, teliti,...","[tim, teliti, indonesia, gabung, murdoch, chil...",24058


In [30]:
# df_combined.to_csv('df_combined.csv', index=False)

In [31]:
df_combined['content_tokens_stopwords_stemmed'] = df_combined['content_tokens_stopwords_stemmed'].apply(lambda tokens: ' '.join(tokens))
df_combined['title_tokens_stopwords_stemmed'] = df_combined['title_tokens_stopwords_stemmed'].apply(lambda tokens: ' '.join(tokens))
df_combined.head(3)

,title,content,title_tokens_stopwords_stemmed,content_tokens_stopwords_stemmed,document_id
0,"Ibu, Perhatikan 5 Hal Ini saat Memilih Car Se...","Bagi orang tua yang memiliki kendaraan, memili...",perhati 5 pilih car seat newborn,orang tua milik kendara memilihcar seat newbo...,0
1,"Catat, Ini 6 Tips Menyimpan Beras agar Tidak ...",Sebagai makanan pokok beras harus disimpan den...,catat 6 tips simpan beras kutu,makan pokok beras simpan kualitas turun sayan...,1
2,"Waspada, Ini 6 Bahaya Asap Rokok bagi Kesehat...",Merokok tidak hanya berbahaya untuk dirimu sen...,waspada 6 bahaya asap rokok sehat anak,rokok bahaya diri orang sekitar bayi anak as...,2


In [32]:
# Melakukan word embedding pada content_tokens_stopwords_stemmed (dokumen/konteks) menggunakan TF-IDF
X_combined = embedding.fit_transform(df_combined['content_tokens_stopwords_stemmed'])

# Melakukan fit/training pada NearestNeighbors
retriever.fit(X_combined)

NearestNeighbors(metric='cosine', n_neighbors=9)

In [33]:
# Melakukan word embedding pada title_tokens_stopwords_stemmed (query) menggunakan TF-IDF
X_ev_combined = embedding.transform(df_combined['title_tokens_stopwords_stemmed'])
X_ev_combined.shape

(24060, 119434)

In [34]:
# Melakukan prediksi dengan menggunakan NearestNeighbors
y_pred_combined = retriever.kneighbors(X_ev_combined, return_distance=False)
y_pred_combined.shape

(24060, 9)

In [35]:
# Menghitung akurasi
acc = top_accuracy(df_combined['document_id'], y_pred_combined)
print('Jumlah Benar\t:', int(acc*len(df_combined['document_id'])), 'dari', len(df_combined['document_id']))
print('Akurasi\t\t:', f'{acc:.4f}')

Jumlah Benar	: 23417 dari 24060
Akurasi		: 0.9733


## Example Use (Implementasi Penggunaan)

In [36]:
# example use get df_combined['content'] based on document_id
query = 'keadaan terbaik'

# Menghilangkan stopwords dan melakukan stemming pada query 
query = [stemmer.stem(word) for word in word_tokenize(query) if word not in stop_words]
query = ' '.join(query)

X_ev = embedding.transform([query])
y_pred = retriever.kneighbors(X_ev, return_distance=False)
y_pred


array([[18268, 18577, 22229, 18604, 15609, 15542, 19682, 22094,  4772]],
      dtype=int64)

In [37]:
# import pickle

# # save embedding and retriever
# pickle.dump(embedding, open('embedding.pkl', 'wb'))
# pickle.dump(retriever, open('retriever.pkl', 'wb'))

In [38]:
selected = df_combined.iloc[y_pred[0][0]]['content']
selected

' Jika industri perfilman punya ajang penghargaan Academy Awards , maka industri pariwisata juga punya ajang bergengsi yang dinamakan World Travel Awards ( WTA ) . Tahun ini , \xa0 WTA 2017 menobatkan sejumlah negara dan perusahaan yang bergerak di industri wisata sebagai yang terbaik dalam beberapa kategori . Galapagos di Ekuador memenangkan kategori Destinasi Pantai Terbaik , sementara Machu Picchu memenangkan kategori Objek Wisata Terbaik . Bandara Internasional Changi di Singapura memenangkan kategori Bandara Terbaik , yang diikuti dengan Singapore Airlines yang memenangkan kategori Maskapai Terbaik . Galapagos terpilih menjadi pemenang karena dianggap memiliki layanan dan fasilitas penunjang kenyamanan turis yang mumpuni . Ditambah lagi dengan pemandangan alamnya yang indah . Kepulauan ini masuk dalam daftar Situs Warisan Dunia UNESCO , karena memiliki 19 pulau yang menyimpan kekayaan bawah laut . Galapagos juga menjadi sumber inspirasi Charles Darwin yang menulis teori evolusi , 

In [39]:
transform_text(embedding, selected)

Text:  Jika industri perfilman punya ajang penghargaan Academy Awards , maka industri pariwisata juga punya ajang bergengsi yang dinamakan World Travel Awards ( WTA ) . Tahun ini ,   WTA 2017 menobatkan sejumlah negara dan perusahaan yang bergerak di industri wisata sebagai yang terbaik dalam beberapa kategori . Galapagos di Ekuador memenangkan kategori Destinasi Pantai Terbaik , sementara Machu Picchu memenangkan kategori Objek Wisata Terbaik . Bandara Internasional Changi di Singapura memenangkan kategori Bandara Terbaik , yang diikuti dengan Singapore Airlines yang memenangkan kategori Maskapai Terbaik . Galapagos terpilih menjadi pemenang karena dianggap memiliki layanan dan fasilitas penunjang kenyamanan turis yang mumpuni . Ditambah lagi dengan pemandangan alamnya yang indah . Kepulauan ini masuk dalam daftar Situs Warisan Dunia UNESCO , karena memiliki 19 pulau yang menyimpan kekayaan bawah laut . Galapagos juga menjadi sumber inspirasi Charles Darwin yang menulis teori evolusi 

# doc2query

In [40]:
# # akan dilakukan pembuatan query baru untuk mengetahui akurasi dari model yang telah dibuat
# # Wikidepia/IndoT5-base-paraphrase

# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# import torch

# model_name = 'doc2query/msmarco-indonesian-mt5-base-v1'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [41]:
# def create_queries(para):
#     input_ids = tokenizer.encode(para, return_tensors='pt')
#     with torch.no_grad():
#         # Here we use Beam-search. It generates better quality queries, but with less diversity
#         beam_outputs = model.generate(
#             input_ids=input_ids, 
#             max_length=64, 
#             num_beams=5, 
#             no_repeat_ngram_size=2, 
#             num_return_sequences=5, 
#             early_stopping=True
#         )
        
#     print("Paragraph:")
#     print(para)

#     print("\nBeam Outputs:")
#     for i in range(len(beam_outputs)):
#         query = tokenizer.decode(beam_outputs[i], skip_special_tokens=True)
#         print(f'{i + 1}: {query}')

    
#     return tokenizer.decode(beam_outputs[0], skip_special_tokens=True)


In [42]:
# text = df['content'][9932]
# (create_queries(text))

In [43]:
# # create query for all documents and save it to a new column
# df['new_query'] = df['content'].apply(create_queries)
# df.head()

# SBERT try

In [44]:
# from sentence_transformers import SentenceTransformer
# from sklearn.metrics.pairwise import cosine_similarity

# # Load pre-trained SBERT model
# model = SentenceTransformer('bert-base-nli-mean-tokens')

# # Preprocess documents and queries
# documents = df['content_tokens_stopwords_stemmed'].apply(lambda tokens: ' '.join(tokens))
# queries = df['title_tokens_stopwords_stemmed'].apply(lambda tokens: ' '.join(tokens))

# # 
# # 
# # get only first 30 row
# documents = documents[:30]
# queries = queries[:30]
# # 
# # 


# # Encode documents and queries
# document_embeddings = model.encode(documents)
# query_embeddings = model.encode(queries)

# # Perform information retrieval
# def retrieve_documents(query, k=5):
#     query_embedding = model.encode([query])
#     similarities = cosine_similarity(query_embedding, document_embeddings)[0]
#     indices = similarities.argsort()[-k:][::-1]  # Retrieve top-k similar documents
#     retrieved_documents = [documents[i] for i in indices]
#     return retrieved_documents



In [45]:
# # Example usage
# query = 'hamil'  # Take the first query
# retrieved_documents = retrieve_documents(query)

# # Print the retrieved documents
# print("Query:", query)
# print("Retrieved Documents:")
# for doc in retrieved_documents:
#     print(doc)